Truy cập để lấy LINK CỬA HÀNG

In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

# Đường dẫn file Excel đầu vào chứa cột "Links"
input_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/Category_Files/category_2.xlsx"

# Đọc file Excel bằng pandas
df = pd.read_excel(input_file)
# Giả sử file Excel có cột "Links" chứa các URL cần truy cập
links = df["Links"].tolist()

# Sử dụng set để lưu các seller link không trùng lặp
seller_links_set = set()

# Khởi tạo Selenium WebDriver (ở đây dùng Chrome)
driver = webdriver.Chrome()
driver.maximize_window()  # Phóng to cửa sổ trình duyệt
driver.implicitly_wait(10)  # Chờ tối đa 10 giây cho các phần tử xuất hiện

# Duyệt qua từng URL trong danh sách
for url in links:
    try:
        print(f"Đang truy cập: {url}")
        driver.get(url)
        # Cho trang load dữ liệu (tùy chỉnh thời gian chờ nếu cần)
        time.sleep(2)
        
        # Tìm phần tử chứa seller link theo cấu trúc HTML được cung cấp.
        # Ta có thể sử dụng CSS selector. Trong HTML mẫu, seller link nằm trong thẻ <a> bên trong div có class:
        # "SellerName__SellerNameStyled-sc-5d1cxl-0 myqYs"
        seller_element = driver.find_element(By.CSS_SELECTOR, "div.SellerName__SellerNameStyled-sc-5d1cxl-0.myqYs a")
        
        # Lấy thuộc tính href của thẻ <a>
        seller_href = seller_element.get_attribute("href")
        print(f"  => Seller link: {seller_href}")
        
        # Thêm vào set để tránh trùng lặp
        seller_links_set.add(seller_href)
    except NoSuchElementException:
        # Nếu không tìm thấy phần tử chứa seller link
        print(f"  => Không tìm thấy seller link trên trang: {url}")
    except Exception as e:
        print(f"  => Lỗi khi xử lý {url}: {e}")

# Đóng trình duyệt
driver.quit()

# Chuyển set thành danh sách để lưu vào file Excel
seller_links_list = list(seller_links_set)

# Lưu kết quả thu thập được vào file Excel mới
output_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/store_seller_links.xlsx"
output_df = pd.DataFrame({"Seller Links": seller_links_list})
output_df.to_excel(output_file, index=False)

print(f"\nĐã lưu danh sách seller link vào file: {output_file}")


Đang truy cập: https://tiki.vn/cu-sac-nhanh-baseus-palm-c-u-20w-for-iphone-ipad-samsung-google-pixel-hang-chinh-hang-p276181269.html?spid=276181277


Truy cập Links cửa hàng để lấy thông tin sản phẩm.

In [42]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, WebDriverException, ElementClickInterceptedException


# Hàm cuộn trang có giới hạn (chừa lại 1/7 trang)
def scroll_page(driver, scroll_count=10, pause=1.5, offset=25):
    """Cuộn trang để tải thêm nội dung."""
    total_height = driver.execute_script("return document.body.scrollHeight")
    scroll_step = (total_height - offset) / scroll_count
    for i in range(1, scroll_count + 1):
        driver.execute_script(f"window.scrollTo(0, {i * scroll_step});")
        time.sleep(pause)
    time.sleep(2)  # Đợi 2 giây sau khi cuộn xong

# Hàm nhấn nút "Xem thêm" nếu còn
def click_view_more(driver):
    """Tìm và nhấn vào nút 'Xem Thêm' nếu có."""
    try:
        while True:
            view_more_button = driver.find_element(By.CSS_SELECTOR, "button.ViewMoreBtn__Wrapper-sc-qs9ydg-0")
            if view_more_button.is_displayed():
                driver.execute_script("arguments[0].click();", view_more_button)
                print("Đã nhấn 'Xem Thêm'. Chờ tải thêm sản phẩm...")
                time.sleep(3)  # Đợi trang tải
            else:
                break
    except (NoSuchElementException, ElementClickInterceptedException):
        print("Không tìm thấy hoặc không thể ấn nút 'Xem Thêm'.")
        return


# Đường dẫn file chứa danh sách cửa hàng
input_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/store_seller_links.xlsx"

# Đọc danh sách seller links
df = pd.read_excel(input_file)
seller_links = df["Seller Links"].dropna().tolist()

# Khởi tạo trình duyệt
driver = webdriver.Chrome()
driver.maximize_window()
driver.implicitly_wait(10)  # Chờ tối đa 10 giây

# Danh sách lưu thông tin
data = []

# Duyệt từng cửa hàng
for link in seller_links:
    try:
        print(f"Đang truy cập: {link}")
        try:
            driver.get(link)
            time.sleep(3)  # Chờ trang tải
        except WebDriverException as e:
            print(f"Lỗi khi mở trang {link}: {e}")
            continue

        # Bước 1: Lấy tên cửa hàng
        try:
            store_name_element = driver.find_element(By.CSS_SELECTOR, "h1.Header__SellerName-sc-vgcg69-2")
            store_name = store_name_element.text.strip()
            # print(f"Tên cửa hàng: {store_name}")
        except NoSuchElementException:
            store_name = None
            print("Không tìm thấy tên cửa hàng.")


        # Bước 1: Click vào tab "Hồ Sơ Cửa Hàng"
        try:
            profile_tab = driver.find_element(By.XPATH, "//div[contains(text(), 'Hồ Sơ Cửa Hàng')]")
            profile_tab.click()
            time.sleep(3)  # Chờ trang tải lại sau khi click
            # print("Đã click vào tab 'Hồ Sơ Cửa Hàng'")
        except NoSuchElementException:
            print("Không tìm thấy tab 'Hồ Sơ Cửa Hàng', bỏ qua trang này.")
            continue

        # Bước 2: Lấy thông tin cửa hàng
        try:
            info_values = driver.find_elements(By.CSS_SELECTOR, 
                "span.StoreInfo__InfoValue-sc-1un24du-5, "
                "span.StoreInfo__InfoLabel-sc-1un24du-4")

            extracted_values = [value.text.strip() for value in info_values if value.text.strip()]
            # print(f"Thông tin cửa hàng: {extracted_values}")
            year = extracted_values[2]
            total_products = extracted_values[5] 
            rating = extracted_values[11][:3]
            review_count = extracted_values[12]
            followers = extracted_values[15] 
            chat_response = extracted_values[18]
        except Exception as e:
            print(f"Lỗi khi lấy thông tin cửa hàng {link}: {e}")
            year, total_products, rating, review_count, followers, chat_response = (None, None, None, None, None, None)

        # Bước 3: Click vào tab "Tất Cả Sản Phẩm"
        try:
            all_products_tab = driver.find_element(By.XPATH, "//div[contains(text(), 'Tất Cả Sản Phẩm')]")
            all_products_tab.click()
            time.sleep(3)  # Chờ trang tải lại sau khi click
            # print("Đã click vào tab 'Tất Cả Sản Phẩm'")
        except NoSuchElementException:
            print("Không tìm thấy tab 'Tất Cả Sản Phẩm', bỏ qua sản phẩm.")
            continue


        # Bước 5: Cuộn trang và nhấn "Xem thêm"
        scroll_page(driver)
        click_view_more(driver)


        # Bước 4: Lấy thông tin từng sản phẩm (Tên, Giá, Số lượng bán)
        try:
            product_elements = driver.find_elements(By.CSS_SELECTOR, "div.ProductRow__Container-sc-13ncpqb-1 a.Product__Wrapper-sc-n99tp2-2")

            for product in product_elements:
                try:
                    product_name_elements = product.find_elements(By.CSS_SELECTOR, "h3")
                    product_name = product_name_elements[0].text if product_name_elements else None

                    sold_elements = product.find_elements(By.CSS_SELECTOR, "span.percent")
                    sold = sold_elements[0].text.replace("Đã bán ", "") if sold_elements else None

                    price_elements = product.find_elements(By.CSS_SELECTOR, "div.Product__PriceDiscountValue-sc-n99tp2-9 span")
                    price = price_elements[0].text if price_elements else None

                    product_link = product.get_attribute("href") if product else None  # Lấy link sản phẩm

                except Exception as e:
                    print(f"Lỗi khi lấy thông tin sản phẩm tại {link}: {e}")
                    product_name, sold, price, product_link = (None, None, None, None)

                # Lưu dữ liệu vào danh sách
                data.append({
                    "Seller Link": link,
                    "Store Name": store_name,
                    "Year Joined": year,
                    "Total Products": total_products,
                    "Rating": rating,
                    "Review Count": review_count,
                    "Followers": followers,
                    "Chat Response Rate": chat_response,
                    "Product Name": product_name,
                    "Product Link": product_link,
                    "Price (VND)": price,
                    "Sold Quantity": sold
                })

        except Exception as e:
            print(f"Lỗi khi lấy danh sách sản phẩm tại {link}: {e}")
            continue

    except Exception as e:
        print(f"Lỗi khi xử lý {link}: {e}")

# Đóng trình duyệt
driver.quit()

# Lưu vào Excel
output_file = r"D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/store_products.xlsx"
output_df = pd.DataFrame(data)
output_df.to_excel(output_file, index=False)

print(f"\nĐã lưu thông tin vào file: {output_file}")


Đang truy cập: https://tiki.vn/cua-hang/vien-thong-ha-nhung?source_screen=product_detail&source_engine=organic
Đã nhấn 'Xem Thêm'. Chờ tải thêm sản phẩm...
Không tìm thấy hoặc không thể ấn nút 'Xem Thêm'.
Đang truy cập: https://tiki.vn/cua-hang/itel-official-store?source_screen=product_detail&source_engine=organic
Không tìm thấy hoặc không thể ấn nút 'Xem Thêm'.

Đã lưu thông tin vào file: D:/du lieu o cu/HUTECH Courses/NCKH(thầy Hường)/E-Commerce-Data-Analysis/Data Scraping Selenium/store_products.xlsx
